In [40]:
import geopandas as gpd
import leafmap
import rasterio 

In [41]:
filepath = 'D:\Trainings\gee\Pabbi_RevisedLandtype-V4.geojson'
gdf=gpd.read_file(filepath)


In [42]:
pure_agri = gdf[gdf['landuse_class'] == 'Pure-Agriculture']
pure_built = gdf[gdf['landuse_class'] == 'Pure-Builtup']
partial_agri = gdf[gdf['landuse_class'] == 'Partial-Agriculture']
partial_built = gdf[gdf['landuse_class'] == 'Partial-Builtup']
mixed = gdf[gdf['landuse_class'] == 'Mixed']


In [43]:
pabbicrop = 'D:\Trainings\gee\pabb_crop_V3.geojson'
landusema_gdf = gpd.read_file(pabbicrop)


In [ ]:
classified_img = 'D:/Trainings/gee/classified_image-V3.tif'

In [44]:
with rasterio.open(classified_img) as src:
    landusema_gdf = landusema_gdf.to_crs(src.crs)
    clipped_image, clipped_transform = rasterio.mask.mask(
        dataset=src,
        shapes=landusema_gdf.geometry,
        crop=True,
        nodata=0,  # or use original nodata if needed
    )
    clipped_meta = src.meta.copy()

# Step 3: Update metadata
clipped_meta.update({
    "height": clipped_image.shape[1],
    "width": clipped_image.shape[2],
    "transform": clipped_transform
})

# Step 4: Save the clipped raster
clipped_path = 'D:/Trainings/gee/classified_image-V3-clipped.tif'
with rasterio.open(clipped_path, 'w', **clipped_meta) as dest:
    dest.write(clipped_image)

print(f"✅ Clipped raster saved to: {clipped_path}")


✅ Clipped raster saved to: D:/Trainings/gee/classified_image-V3-clipped.tif


In [45]:
import leafmap

# 1. Define the mapping of raster values → colors
#    Make sure these integer codes match how your RF labels were encoded:
#    0: Pure-Agriculture, 1: Partial-Agriculture, 2: Pure-Builtup,
#    3: Partial-Builtup, 4: Mixed

colorlist = ['lightgreen','lightblue','darkgreen','darkblue']
mycolors = {
    1: '#006400',    # Pure-Agriculture
    2: '#90EE90',   # Partial-Agriculture
    3: '#00008B',     # Pure-Builtup
    4: '#ADD8E6',    # Partial-Builtup
   # 5: '#FF0000',           # Mixed
}
classified_clipped='D:/Trainings/gee/classified_image-V2-clipped.tif'

m = leafmap.Map(center=(37.5, 70), zoom=6)
m.add_basemap('Google Satellite')

# 2. Add the classified raster with your discrete colormap
m.add_raster(
    classified_clipped,
    layer_name='Pabbi Revised Landtype',
    colormap=colorlist,     
    opacity=0.5,
    vmin=0,
    vmax=4,
    layer_opacity=0.5
)

# 2. Manually add a legend for those codes
legend_dict = {
    'Pure-Agriculture':   '#006400',
    'Partial-Agriculture':'#90EE90',
    'Pure-Builtup':       '#00008B',
    'Partial-Builtup':    '#ADD8E6',
  #  'Mixed':              '#FF0000',
}

# Adding landusema_gdf to the map
m.add_gdf(
    landusema_gdf,
    layer_name='Boundaries',
    #color='landuse_class',
    #palette=legend_dict,
    opacity=0.3,
    legend=True
)

m.add_legend(
    legend_title='Landuse Classification',
    legend_dict=legend_dict,
    position='bottomleft'
)
m.add_layer_manager()
m


Map(center=[34.016954, 71.7731905], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…